In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Configuration
target_date_str = "20251227"
input_file = "E:\\python_env\\Incident Ticket_20260131105108.xlsx"
lookup_file = "SubRootcuz.xlsx"
output_file = "E:\\python_env\\result.xlsx"

def process_ticket_data_no_drop(input_file, target_date_str):
    """
    Process ticket data without dropping any columns.
    
    Args:
        input_file (str): Path to input Excel file
        target_date_str (str): Target date in format YYYYMMDD
    
    Returns:
        pd.DataFrame: Processed dataframe
    """
    
    # 1. Read data from Excel file
    print(f"Reading data from {input_file}...")
    try:
        df = pd.read_excel(input_file)
        print(f"Successfully loaded {len(df)} rows and {len(df.columns)} columns")
    except Exception as e:
        print(f"Error reading file: {e}")
        return None
    
    # 2. Parse target date
    try:
        target_date = datetime.strptime(target_date_str, "%Y%m%d")
        target_date_start = target_date.replace(hour=0, minute=0, second=0, microsecond=0)
        target_date_end = target_date.replace(hour=23, minute=59, second=59, microsecond=0)
        print(f"Target date: {target_date_start} to {target_date_end}")
    except ValueError:
        print(f"Error: Invalid date format. Expected YYYYMMDD, got {target_date_str}")
        return None
    
    # 3. Process Fault First Occur Time
    if "Fault First Occur Time" in df.columns:
        # Convert to datetime if needed
        df["Fault First Occur Time"] = pd.to_datetime(df["Fault First Occur Time"], errors='coerce')
        
        # Apply the rule: if before target date, set to 00:00:00 of target date
        mask_before = df["Fault First Occur Time"] < target_date_start
        df.loc[mask_before, "Fault First Occur Time"] = target_date_start
        
        print(f"Adjusted {mask_before.sum()} 'Fault First Occur Time' entries to {target_date_start}")
    else:
        print("Warning: 'Fault First Occur Time' column not found")
    
    # 4. Process Fault Recovery Time
    if "Fault Recovery Time(Process TT_faultrecoverytime)" in df.columns:
        # Convert to datetime
        df["Fault Recovery Time(Process TT_faultrecoverytime)"] = pd.to_datetime(
            df["Fault Recovery Time(Process TT_faultrecoverytime)"], errors='coerce'
        )
        
        # Apply the rule: if after target date or NaN, set to 23:59:59 of target date
        mask_after = df["Fault Recovery Time(Process TT_faultrecoverytime)"] > target_date_end
        mask_nan = df["Fault Recovery Time(Process TT_faultrecoverytime)"].isna()
        
        df.loc[mask_after | mask_nan, "Fault Recovery Time(Process TT_faultrecoverytime)"] = target_date_end
        
        print(f"Adjusted {mask_after.sum()} entries for being after target date")
        print(f"Adjusted {mask_nan.sum()} entries for being NaN")
    else:
        print("Warning: 'Fault Recovery Time(Process TT_faultrecoverytime)' column not found")
    
    # 5. Calculate MTTR (only if both time columns exist)
    if "Fault First Occur Time" in df.columns and "Fault Recovery Time(Process TT_faultrecoverytime)" in df.columns:
        df["MTTR"] = df["Fault Recovery Time(Process TT_faultrecoverytime)"] - df["Fault First Occur Time"]
        
        # Cap MTTR at 23:59:59 if needed
        max_duration = timedelta(hours=23, minutes=59, seconds=59)
        df["MTTR"] = df["MTTR"].apply(lambda x: min(x, max_duration) if pd.notnull(x) else x)
        
        # Convert MTTR to string format HH:MM:SS
        def format_timedelta(td):
            if pd.isnull(td):
                return "00:00:00"
            total_seconds = int(td.total_seconds())
            hours = total_seconds // 3600
            minutes = (total_seconds % 3600) // 60
            seconds = total_seconds % 60
            return f"{hours:02d}:{minutes:02d}:{seconds:02d}"
        
        df["MTTR"] = df["MTTR"].apply(format_timedelta)
        print(f"MTTR calculated for {len(df)} rows")
        
        # Calculate MTTR_sec (MTTR in seconds)
        def timedelta_to_seconds(td):
            if pd.isnull(td):
                return 0
            return int(td.total_seconds())
        
        # Create a timedelta version for calculation
        df["MTTR_timedelta"] = pd.to_timedelta(df["MTTR"])
        df["MTTR_sec"] = df["MTTR_timedelta"].apply(timedelta_to_seconds)
        print(f"MTTR_sec calculated for {len(df)} rows")
        
        # Drop the intermediate column
        df = df.drop(columns=["MTTR_timedelta"])
    else:
        print("Warning: Cannot calculate MTTR - missing time columns")
        df["MTTR"] = "00:00:00"
        df["MTTR_sec"] = 0
    
    # 6. Create Aggregated_RC column
    if "Root Cause(Process TT_root_cause)" in df.columns and "Probable Cause(Process TT_possiblecause)" in df.columns:
        df["Aggregated_RC"] = df.apply(
            lambda row: row["Root Cause(Process TT_root_cause)"] 
            if pd.notnull(row["Root Cause(Process TT_root_cause)"]) 
            else row["Probable Cause(Process TT_possiblecause)"],
            axis=1
        )
    else:
        print("Warning: Required columns for Aggregated_RC not found")
        df["Aggregated_RC"] = ""
    
    # 7. Create Aggregated_SC column
    if "Sub Root Cause(Process TT_sub_root_cause)" in df.columns and "Probable Sub Cause(Process TT_probsubcause)" in df.columns:
        df["Aggregated_SC"] = df.apply(
            lambda row: row["Sub Root Cause(Process TT_sub_root_cause)"] 
            if pd.notnull(row["Sub Root Cause(Process TT_sub_root_cause)"]) 
            else row["Probable Sub Cause(Process TT_probsubcause)"],
            axis=1
        )
    else:
        print("Warning: Required columns for Aggregated_SC not found")
        df["Aggregated_SC"] = ""
    
    # 7.5 Fill empty Aggregated_RC and Aggregated_SC values
    print("\nFilling empty Aggregated_RC and Aggregated_SC values...")
    
    # Function to check if a value is empty (NaN, None, or empty string)
    def is_empty(value):
        return pd.isna(value) or value is None or str(value).strip() == ""
    
    # Fill empty Aggregated_RC with "BSS - Under Investigation"
    empty_rc_count = df["Aggregated_RC"].apply(is_empty).sum()
    df["Aggregated_RC"] = df["Aggregated_RC"].apply(
        lambda x: "BSS - Under Investigation" if is_empty(x) else x
    )
    print(f"Filled {empty_rc_count} empty Aggregated_RC values with 'BSS - Under Investigation'")
    
    # Fill empty Aggregated_SC with "Unclear"
    empty_sc_count = df["Aggregated_SC"].apply(is_empty).sum()
    df["Aggregated_SC"] = df["Aggregated_SC"].apply(
        lambda x: "Unclear" if is_empty(x) else x
    )
    print(f"Filled {empty_sc_count} empty Aggregated_SC values with 'Unclear'")
    
    # 8. Calculate outage columns
    print("\nCalculating outage columns...")
    
    # Define column name mappings
    column_mappings = {
        "2G_outage": "2G-Affected Site Count(Create TT_sitecount2g)",
        "3G_outage": "3G-Affected Site Count(Create TT_sitecount3g)",
        "4G_outage": "4G-Affected Site Count(Create TT_sitecount4g)",
        "TDD_outage": "TDD Affected Site Count(Create TT_tddsitecount)",
        "5G_outage": "5G-Affected Site Count(Create TT_sitecount5g)"
    }
    
    # Calculate each outage column
    for outage_col, count_col in column_mappings.items():
        if count_col in df.columns:
            # Convert to numeric, filling NaN with 0
            df[count_col] = pd.to_numeric(df[count_col], errors='coerce').fillna(0).astype(int)
            df[outage_col] = df["MTTR_sec"] * df[count_col]
            print(f"Created {outage_col} using {count_col}")
        else:
            # If count column doesn't exist, set outage to 0
            df[outage_col] = 0
            print(f"Warning: {count_col} not found. Setting {outage_col} to 0")
    
    # 9. Drop rows where Aggregated_SC is exactly "ER"
    initial_count = len(df)
    if "Aggregated_SC" in df.columns:
        # Convert to string for comparison, then check for exact match with "ER"
        df = df[df["Aggregated_SC"].astype(str).str.strip() != "ER"]
        removed_er_count = initial_count - len(df)
        print(f"Removed {removed_er_count} rows where Aggregated_SC is 'ER'")
    else:
        print("Warning: 'Aggregated_SC' column not found, cannot filter by 'ER'")
    
    # 10. Drop rows where Site Province is empty (if column exists)
    if "Site Province(Create TT_province)" in df.columns:
        initial_count = len(df)
        df = df.dropna(subset=["Site Province(Create TT_province)"])
        df = df[df["Site Province(Create TT_province)"].astype(str).str.strip() != ""]
        removed_province_count = initial_count - len(df)
        print(f"Removed {removed_province_count} rows with empty Site Province")
    else:
        print("Warning: 'Site Province(Create TT_province)' column not found")
    
    return df

# Main processing
if __name__ == "__main__":
    print("Starting processing...")
    print(f"Input file: {input_file}")
    print(f"Lookup file: {lookup_file}")
    print(f"Output file: {output_file}")
    print(f"Target date: {target_date_str}")
    
    # Process the ticket data
    df_result = process_ticket_data_no_drop(input_file, target_date_str)
    
    if df_result is not None:
        # Load lookup dataframe
        print(f"\nLoading lookup data from {lookup_file}...")
        try:
            lookup_df = pd.read_excel(lookup_file, sheet_name='Sheet1')
            print(f"Successfully loaded lookup table with {len(lookup_df)} rows")
        except Exception as e:
            print(f"Error reading lookup file: {e}")
            # If lookup fails, just use the processed df as final
            df_final = df_result
        else:
            # Merge df_result with lookup_df
            print("Merging with lookup data...")
            merged_df = pd.merge(
                df_result,
                lookup_df[['RC', 'SC', 'CATEGORY', 'MS/NON MS']],
                left_on=['Aggregated_RC', 'Aggregated_SC'],
                right_on=['RC', 'SC'],
                how='left'
            )
            
            # Drop the redundant RC and SC columns from the lookup
            merged_df = merged_df.drop(columns=['RC', 'SC'])
            df_final = merged_df
        
        # Reorder columns to have new columns at the end for easier viewing
        all_columns = df_final.columns.tolist()
        new_columns = [
            "MTTR", "MTTR_sec", "Aggregated_RC", "Aggregated_SC",
            "2G_outage", "3G_outage", "4G_outage", "TDD_outage", "5G_outage"
        ]
        
        # Add CATEGORY and MS/NON MS if they exist
        if "CATEGORY" in df_final.columns:
            new_columns.append("CATEGORY")
        if "MS/NON MS" in df_final.columns:
            new_columns.append("MS/NON MS")
        
        other_columns = [col for col in all_columns if col not in new_columns]
        final_columns = other_columns + new_columns
        df_final = df_final[final_columns]
        
        # Save to Excel
        try:
            df_final.to_excel(output_file, index=False)
            print(f"\nSuccessfully saved processed data to {output_file}")
            print(f"Final dataset shape: {df_final.shape}")
            print(f"Total columns: {len(df_final.columns)}")
            
            # Show summary statistics
            print("\nSummary statistics for new columns:")
            if "MTTR_sec" in df_final.columns:
                print(f"MTTR_sec - Min: {df_final['MTTR_sec'].min()}s, Max: {df_final['MTTR_sec'].max()}s, Avg: {df_final['MTTR_sec'].mean():.2f}s")
            
            # Show outage column totals
            outage_columns = ["2G_outage", "3G_outage", "4G_outage", "TDD_outage", "5G_outage"]
            for col in outage_columns:
                if col in df_final.columns:
                    total = df_final[col].sum()
                    print(f"{col}: Total = {total:,} site-seconds")
            
        except Exception as e:
            print(f"Error saving file: {e}")
    
    print("\nProcessing complete!")

Starting processing...
Input file: E:\python_env\Incident Ticket_20260131105108.xlsx
Lookup file: SubRootcuz.xlsx
Output file: E:\python_env\result.xlsx
Target date: 20251227
Reading data from E:\python_env\Incident Ticket_20260131105108.xlsx...


C:\Users\yahya.f\AppData\Local\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Successfully loaded 2252 rows and 34 columns
Target date: 2025-12-27 00:00:00 to 2025-12-27 23:59:59
Adjusted 473 'Fault First Occur Time' entries to 2025-12-27 00:00:00
Adjusted 428 entries for being after target date
Adjusted 87 entries for being NaN
MTTR calculated for 2252 rows
MTTR_sec calculated for 2252 rows

Filling empty Aggregated_RC and Aggregated_SC values...
Filled 0 empty Aggregated_RC values with 'BSS - Under Investigation'
Filled 2 empty Aggregated_SC values with 'Unclear'

Calculating outage columns...
Created 2G_outage using 2G-Affected Site Count(Create TT_sitecount2g)
Created 3G_outage using 3G-Affected Site Count(Create TT_sitecount3g)
Created 4G_outage using 4G-Affected Site Count(Create TT_sitecount4g)
Created TDD_outage using TDD Affected Site Count(Create TT_tddsitecount)
Created 5G_outage using 5G-Affected Site Count(Create TT_sitecount5g)
Removed 95 rows where Aggregated_SC is 'ER'
Removed 1 rows with empty Site Province

Loading lookup data from SubRootcuz.x

In [3]:
# =========================
# NEW PART: TCHA & OUTAGE SUMMARY DATAFRAMES
# =========================

# User-defined TCHA values
TCHA_VALUES = {
    "2G": 98.56,
    "3G": 96.56,
    "4G": 98.05,
    "5G": 98.16,
    "TDD": 90.00
}

# Categories to aggregate
CATEGORIES = ["power", "NOA", "Non Telecom", "PA / UA", "Spare", "Third Party"]

# Technology to outage column mapping
TECH_OUTAGE_COLUMNS = {
    "2G": "2G_outage",
    "3G": "3G_outage",
    "4G": "4G_outage",
    "TDD": "TDD_outage",
    "5G": "5G_outage"
}

def build_final_tech_summary(tech, outage_col):
    """
    Build final technology summary with exclusion column and proper formatting.
    
    Args:
        tech (str): Technology name (e.g., "2G", "3G", etc.)
        outage_col (str): Outage column name in df_final
    
    Returns:
        pd.DataFrame: Final technology summary DataFrame
    """
    summary_rows = []
    
    # Total outage (will be removed later)
    total_outage = df_final[outage_col].sum()
    summary_rows.append({
        "Metric": f"{tech}_outage_total",
        "Value": total_outage
    })
    
    # Outage per category
    category_outages = {}
    for category in CATEGORIES:
        category_value = df_final.loc[
            df_final["CATEGORY"].astype(str).str.strip().str.lower() == category.lower(),
            outage_col
        ].sum()
        category_outages[category] = category_value
        summary_rows.append({
            "Metric": f"{tech}_outage_{category.replace(' ', '_')}",
            "Value": category_value
        })
    
    # TCHA value
    tcha_value = TCHA_VALUES[tech]
    summary_rows.append({
        "Metric": f"{tech}_TCHA",
        "Value": tcha_value
    })
    
    # Convert to DataFrame
    df = pd.DataFrame(summary_rows)
    
    # Add exclusion column and initialize with NaN
    df['exclusion'] = None
    
    # Calculate exclusion for each category
    for category in CATEGORIES:
        category_key = category.replace(' ', '_')
        metric_name = f'{tech}_outage_{category_key}'
        
        # Find the row for this category
        mask = df['Metric'] == metric_name
        
        if mask.any():
            category_outage = category_outages[category]
            
            # Avoid division by zero
            if total_outage > 0:
                exclusion_value = tcha_value + (category_outage * (100 - tcha_value) / total_outage)
                # Round to 2 decimal places
                exclusion_value = round(exclusion_value, 2)
            else:
                exclusion_value = tcha_value
            
            df.loc[mask, 'exclusion'] = exclusion_value
    
    # Remove the total row (e.g., "2G_outage_total", "3G_outage_total", etc.)
    df = df[~df['Metric'].str.contains('_outage_total$')].copy()
    
    # Fill empty exclusion values with Value column data (for TCHA row)
    df['exclusion'] = df.apply(
        lambda row: row['Value'] if pd.isna(row['exclusion']) else row['exclusion'],
        axis=1
    )
    
    # Format the date for column renaming
    try:
        report_date = datetime.strptime(target_date_str, "%Y%m%d").strftime("%Y-%m-%d")
    except:
        report_date = target_date_str  # fallback to original format
    
    # Rename exclusion column to report date
    df = df.rename(columns={'exclusion': report_date})
    
    # Drop the Value column
    df = df.drop(columns=['Value'])
    
    return df

# Create final DataFrames per technology
df_2G_outage_summary  = build_final_tech_summary("2G",  TECH_OUTAGE_COLUMNS["2G"])
df_3G_outage_summary  = build_final_tech_summary("3G",  TECH_OUTAGE_COLUMNS["3G"])
df_4G_outage_summary  = build_final_tech_summary("4G",  TECH_OUTAGE_COLUMNS["4G"])
df_TDD_outage_summary = build_final_tech_summary("TDD", TECH_OUTAGE_COLUMNS["TDD"])
df_5G_outage_summary  = build_final_tech_summary("5G",  TECH_OUTAGE_COLUMNS["5G"])

# Process each DataFrame: put TCHA row at top and append "_exclusion" to category rows
def process_tech_dataframe(df, tech):
    """
    Process technology DataFrame to:
    1. Put TCHA row at the top
    2. Append "_exclusion" to category rows in Metric column
    3. Return processed DataFrame
    """
    # Get the TCHA row
    tcha_mask = df['Metric'] == f'{tech}_TCHA'
    tcha_row = df[tcha_mask]
    
    # Get category rows (all other rows)
    category_rows = df[~tcha_mask].copy()
    
    # Append "_exclusion" to category rows' Metric values
    category_rows['Metric'] = category_rows['Metric'] + '_exclusion'
    
    # Format the date for column renaming (already done in build_final_tech_summary)
    # Get report date from column name (the column that's not 'Metric')
    value_column = [col for col in df.columns if col != 'Metric'][0]
    
    # Reorder: TCHA row first, then category rows
    processed_df = pd.concat([tcha_row, category_rows], ignore_index=True)
    
    return processed_df

# Process all technology DataFrames
df_2G_outage_summary  = process_tech_dataframe(df_2G_outage_summary, "2G")
df_3G_outage_summary  = process_tech_dataframe(df_3G_outage_summary, "3G")
df_4G_outage_summary  = process_tech_dataframe(df_4G_outage_summary, "4G")
df_TDD_outage_summary = process_tech_dataframe(df_TDD_outage_summary, "TDD")
df_5G_outage_summary  = process_tech_dataframe(df_5G_outage_summary, "5G")

# Print summary to verify
print("\nFinal processed summary DataFrames:")
for tech_df, tech_name in [
    (df_2G_outage_summary, "2G"),
    (df_3G_outage_summary, "3G"),
    (df_4G_outage_summary, "4G"),
    (df_TDD_outage_summary, "TDD"),
    (df_5G_outage_summary, "5G")
]:
    print(f"\n{tech_name} Summary ({len(tech_df)} rows):")
    print(tech_df.to_string(index=False))
    print("-" * 50)


Final processed summary DataFrames:

2G Summary (7 rows):
                         Metric  2025-12-27
                        2G_TCHA       98.56
      2G_outage_power_exclusion       99.39
        2G_outage_NOA_exclusion       98.56
2G_outage_Non_Telecom_exclusion       98.75
    2G_outage_PA_/_UA_exclusion       98.66
      2G_outage_Spare_exclusion       98.56
2G_outage_Third_Party_exclusion       98.64
--------------------------------------------------

3G Summary (7 rows):
                         Metric  2025-12-27
                        3G_TCHA       96.56
      3G_outage_power_exclusion       98.62
        3G_outage_NOA_exclusion       96.56
3G_outage_Non_Telecom_exclusion       97.00
    3G_outage_PA_/_UA_exclusion       96.79
      3G_outage_Spare_exclusion       96.56
3G_outage_Third_Party_exclusion       96.73
--------------------------------------------------

4G Summary (7 rows):
                         Metric  2025-12-27
                        4G_TCHA       98.05
   